#   Tarea 2 pregunta 1


##  Importacion de bibliotecas

In [19]:
import os

import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import tensorflow as tf
from tensorflow import keras    
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

##  Extraccion de datos y tratamiento

In [14]:
text_file_path="spa.txt"
with open(text_file_path) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [15]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]


In [16]:
# Limpiar texto
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size      = 15_000
sequence_length = 20
batch_size      = 64

def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

# Vectorizar
eng_vectorization = TextVectorization(
    max_tokens  = vocab_size,
    output_mode = "int",
    output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

In [17]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "english": eng,
            "spanish": spa[:, :-1],
        },
        spa[:, 1:],
    )

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds   = make_dataset(val_pairs)

In [18]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["english"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["spanish"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


##  Hiperparametros

In [21]:
embed_dim = 256 
dense_dim = 2048 
num_heads = 8 

##  Modelo

In [20]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(  input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)  
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    
    def call(self,inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions 
    
    def compute_mask(self, inputs, mask=None):   
        return tf.math.not_equal(inputs, 0)

    def get_config(self):   
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,})  
        return config

In [22]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim          
        self.dense_dim = dense_dim        
        self.num_heads = num_heads        
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),
                                            layers.Dense(embed_dim),])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):   
        if mask is not None:                 
            mask = mask[:, tf.newaxis, :]    
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input  = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):   
        config = super().get_config()
        config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
        })
        return config

In [23]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential( [layers.Dense(dense_dim, activation="relu"),
                                             layers.Dense(embed_dim),])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True 
    def get_config(self):
        config = super().get_config()
        config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
        })
        return config
    
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")  
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))    
        mult = tf.concat(         
            [tf.expand_dims(batch_size, -1),              
            tf.constant([1, 1], dtype=tf.int32)], axis=0)         
        return tf.tile(mask, mult)
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs) 
        if mask is not None:   
            padding_mask = tf.cast(   
                mask[:, tf.newaxis, :], dtype="int32")   
            padding_mask = tf.minimum(padding_mask, causal_mask)  
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask) 
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,   
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [24]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x) 

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs) 
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)  

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [26]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['english[0][0]']                
 alEmbedding)                                                                                     
                                                                                                  
 positional_embedding_1 (Positi  (None, None, 256)   3845120     ['spanish[0][0]']            

##  Testing

In [27]:
EPOCAS = 10
transformer.fit(train_ds, epochs=EPOCAS, validation_data=val_ds)

Epoch 1/10
1302/1302 [==============================] - 159s 116ms/step - loss: 1.6823 - accuracy: 0.4142 - val_loss: 1.3289 - val_accuracy: 0.5040
Epoch 2/10
1302/1302 [==============================] - 149s 114ms/step - loss: 1.3495 - accuracy: 0.5285 - val_loss: 1.1970 - val_accuracy: 0.5523
Epoch 3/10
1302/1302 [==============================] - 150s 115ms/step - loss: 1.1940 - accuracy: 0.5786 - val_loss: 1.0899 - val_accuracy: 0.5982
Epoch 4/10
1302/1302 [==============================] - 174s 133ms/step - loss: 1.1028 - accuracy: 0.6116 - val_loss: 1.0440 - val_accuracy: 0.6194
Epoch 5/10
1302/1302 [==============================] - 250s 192ms/step - loss: 1.0544 - accuracy: 0.6335 - val_loss: 1.0164 - val_accuracy: 0.6353
Epoch 6/10
1302/1302 [==============================] - 251s 193ms/step - loss: 1.0237 - accuracy: 0.6484 - val_loss: 1.0121 - val_accuracy: 0.6382
Epoch 7/10
1302/1302 [==============================] - 250s 192ms/step - loss: 1.0031 - accuracy: 0.6602 - val_

In [28]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence         = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer(
            {
                "english": tokenized_input_sentence,
                "spanish": tokenized_target_sentence,
            }
        )

        #sampled_token_index = tf.convert_to_numpy(tf.argmax(predictions[0, i, :])).item(0)
        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

In [29]:
test_eng_texts = [pair[0] for pair in test_pairs]
# almacenar un diccionario de predicciones
predictions_dict = {}
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated     = decode_sequence(input_sentence)
    predictions_dict[input_sentence] = translated

In [30]:
predictions_dict


{"Aren't you in town?": '[start] no estás en la ciudad [end]',
 'I am going to stay here till the day after tomorrow.': '[start] mañana me voy a estar aquí hasta el día [end]',
 'Where did you see the boy?': '[start] dónde viste el niño [end]',
 "I'm beginning to lose my patience.": '[start] estoy pasando que me [UNK] la mala [end]',
 'I need one more favor.': '[start] necesito un favor más [end]',
 'You have changed so much that I can hardly recognize you.': '[start] has tenido tanto que puedo a las [UNK] que te lo [UNK] [end]',
 'Send Tom a message.': '[start] [UNK] a tom un mensaje [end]',
 "That's all we know.": '[start] es todo lo que nosotros [end]',
 "Look! There's a cat in the kitchen.": '[start] mira hay un gato en la cocina [end]',
 'I have a terrible pain.': '[start] tengo un dolor de cabeza [end]',
 'Tom grew up with Mary and John.': '[start] tom se fue a mary con john [end]',
 'Forgiveness is the greatest gift of all.': '[start] [UNK] es el mejor libro de verdad [end]',
 '